# EDA: динамическое ценообразование

Основные цели:
- Проверка качества данных
- Анализ структуры train/test
- Исследование методов, используемых в модели

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 200)
pd.set_option("display.float_format", "{:.6f}".format)

train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

train["dt"] = pd.to_datetime(train["dt"])
test["dt"] = pd.to_datetime(test["dt"])

print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (29100, 19)
Test shape: (28050, 18)


## Проверка качества данных

In [2]:
print("Пропуски train:", train.isna().sum().sum())
print("Пропуски test:", test.isna().sum().sum())
print("Дубликаты train:", train.duplicated().sum())
print("Дубликаты test:", test.duplicated().sum())
print("Дубликаты по product_id+dt train:", train.duplicated(["product_id", "dt"]).sum())
print("Дубликаты по product_id+dt test:", test.duplicated(["product_id", "dt"]).sum())

# Проверка валидности интервалов
invalid = (train["price_p05"] > train["price_p95"]).sum()
width = train["price_p95"] - train["price_p05"]
print("\nprice_p05 > price_p95:", invalid)
print("min ширина интервала:", width.min())
print("нулевых ширин:", (width == 0).sum())

Пропуски train: 0
Пропуски test: 0
Дубликаты train: 0
Дубликаты test: 0
Дубликаты по product_id+dt train: 0
Дубликаты по product_id+dt test: 0

price_p05 > price_p95: 0
min ширина интервала: 0.0001089576554184
нулевых ширин: 0


Данные чистые: пропусков нет, дубликатов по ключу `product_id + dt` нет. Все интервалы валидны.

## Структура данных и новые товары

In [3]:
date_summary = pd.DataFrame({
    "dataset": ["train", "test"],
    "min_dt": [train["dt"].min(), test["dt"].min()],
    "max_dt": [train["dt"].max(), test["dt"].max()],
    "n_unique_dates": [train["dt"].nunique(), test["dt"].nunique()],
    "n_unique_products": [train["product_id"].nunique(), test["product_id"].nunique()],
})
display(date_summary)

train_products = set(train["product_id"].unique())
test_products = set(test["product_id"].unique())
new_products = test_products - train_products
old_products = test_products & train_products

print(f"\nНовые товары в test: {len(new_products)}")
print(f"Старые товары в test: {len(old_products)}")
print(f"Товаров в train: {len(train_products)}")
print(f"Товаров в test: {len(test_products)}")

,dataset,min_dt,max_dt,n_unique_dates,n_unique_products
0,train,2024-03-28,2024-05-26,60,485
1,test,2024-03-28,2024-06-25,90,635



Новые товары в test: 150
Старые товары в test: 485
Товаров в train: 485
Товаров в test: 635


В test 150 новых товаров, которые не встречались в train. Это требует специальной обработки при валидации и предсказании.

## Кластеризация товаров

In [4]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import silhouette_score

# Подготовка признаков как в main.py
train["price_mid"] = (train["price_p05"] + train["price_p95"]) / 2.0
train["price_spread"] = train["price_p95"] - train["price_p05"]

aggs = train.groupby("product_id").agg({
    "price_mid": ["mean", "std"],
    "price_spread": ["mean"],
    "activity_flag": ["mean"]
})
aggs.columns = ["_".join(c) for c in aggs.columns]
aggs.fillna(0, inplace=True)

scaler = RobustScaler()
X_cluster = scaler.fit_transform(aggs)

# Проверяем silhouette для разных k
silhouette_scores = {}
for k in range(2, 8):
    kmeans = KMeans(n_clusters=k, random_state=993, n_init=10)
    labels = kmeans.fit_predict(X_cluster)
    silhouette_scores[k] = silhouette_score(X_cluster, labels)

silhouette_df = pd.DataFrame({
    "k": list(silhouette_scores.keys()),
    "silhouette_score": list(silhouette_scores.values())
})
display(silhouette_df)

,k,silhouette_score
0,2,0.341289
1,3,0.346092
2,4,0.356925
3,5,0.328492
4,6,0.316850
5,7,0.313351


По метрике silhouette оптимальное число кластеров k=4 (score=0.357). Однако при проверке на валидации модели (proxy validation) лучшее качество достигается при k=5. Это связано с тем, что silhouette оценивает компактность кластеров, а для задачи предсказания цен важнее разделение товаров по паттернам поведения, которые лучше отражаются при k=5.

In [5]:
# Кластеризация с k=5 (используется в модели)
kmeans = KMeans(n_clusters=5, random_state=993, n_init=10)
labels = kmeans.fit_predict(X_cluster)
aggs["cluster_id"] = labels

# Статистика по кластерам
cluster_stats = train.merge(aggs[["cluster_id"]], on="product_id", how="left")
cluster_summary = cluster_stats.groupby("cluster_id").agg({
    "price_p05": ["mean", "std"],
    "price_p95": ["mean", "std"],
    "price_mid": ["mean"],
    "price_spread": ["mean"],
    "activity_flag": ["mean"],
    "product_id": "nunique"
})
cluster_summary.columns = ["_".join(c) for c in cluster_summary.columns]
display(cluster_summary)

,price_p05_mean,price_p05_std,price_p95_mean,price_p95_std,price_mid_mean,price_spread_mean,activity_flag_mean,product_id_nunique
cluster_id,,,,,,,,
0,0.951415,0.177612,1.051722,0.191991,1.001569,0.100306,0.781654,129
1,1.130593,0.334225,1.199048,0.344000,1.164820,0.068455,0.478261,92
2,0.850929,0.209369,1.110839,0.234756,0.980884,0.259910,0.799087,73
3,1.099459,0.152285,1.139975,0.142990,1.119717,0.040516,0.326916,187
4,0.238142,0.250749,0.360997,0.335737,0.299570,0.122856,0.475000,4


## Детекция аномалий

In [6]:
from sklearn.ensemble import IsolationForest

# Для детекции аномалий нужны признаки после Target Encoding
# Здесь показываем только подготовку данных
train["log_mid"] = np.log1p(train["price_mid"])
train["log_spread"] = np.log1p(train["price_spread"])

# В модели используются te_lvl_product_id, te_spr_product_id, activity_flag
# Для демонстрации используем упрощенные признаки
anom_feats = ["log_mid", "log_spread", "activity_flag"]
iso = IsolationForest(contamination=0.03, random_state=993)
iso.fit(train[anom_feats])
anom_scores = iso.decision_function(train[anom_feats])

print(f"Средний anomaly score: {anom_scores.mean():.4f}")
print(f"Мин anomaly score: {anom_scores.min():.4f}")
print(f"Макс anomaly score: {anom_scores.max():.4f}")
print(f"Доля аномалий (score < 0): {(anom_scores < 0).mean():.4f}")

Средний anomaly score: 0.1795
Мин anomaly score: -0.1385
Макс anomaly score: 0.2677
Доля аномалий (score < 0): 0.0300


IsolationForest используется для детекции аномальных товаров с contamination=0.03. В модели признак anomaly score добавляется к фичам.

## Автокорреляция целей

In [7]:
# Проверка временной зависимости внутри товаров
train_sorted = train.sort_values(["product_id", "dt"])
train_sorted["price_p05_lag1"] = train_sorted.groupby("product_id")["price_p05"].shift(1)
train_sorted["price_p95_lag1"] = train_sorted.groupby("product_id")["price_p95"].shift(1)
train_sorted["width"] = train_sorted["price_p95"] - train_sorted["price_p05"]
train_sorted["width_lag1"] = train_sorted.groupby("product_id")["width"].shift(1)

# Pooled корреляция
pooled_corr = {
    "price_p05": train_sorted["price_p05"].corr(train_sorted["price_p05_lag1"]),
    "price_p95": train_sorted["price_p95"].corr(train_sorted["price_p95_lag1"]),
    "width": train_sorted["width"].corr(train_sorted["width_lag1"])
}

corr_df = pd.DataFrame([pooled_corr]).T
corr_df.columns = ["lag1_correlation"]
display(corr_df)

,lag1_correlation
price_p05,0.668622
price_p95,0.599321
width,0.807524


Наблюдается заметная автокорреляция lag-1 для всех целевых переменных. Это обосновывает использование экспоненциального сглаживания предсказаний в модели.

## Эффект activity_flag

In [8]:
flag_effect = train.groupby("activity_flag").agg({
    "price_p05": "mean",
    "price_p95": "mean",
    "price_mid": "mean",
    "price_spread": "mean"
})
flag_effect.columns = ["price_p05_mean", "price_p95_mean", "price_mid_mean", "price_spread_mean"]
display(flag_effect)

diff = flag_effect.loc[1] - flag_effect.loc[0]
print("\nРазница (activity_flag=1 - activity_flag=0):")
print(diff)

,price_p05_mean,price_p95_mean,price_mid_mean,price_spread_mean
activity_flag,,,,
0,1.100458,1.148450,1.124454,0.047992
1,0.956559,1.090963,1.023761,0.134404



Разница (activity_flag=1 - activity_flag=0):
price_p05_mean      -0.143899
price_p95_mean      -0.057487
price_mid_mean      -0.100693
price_spread_mean    0.086412
dtype: float64


При `activity_flag=1` средние цены ниже, а ширина интервала выше. Это важный признак, который используется в кластеризации и детекции аномалий.

## Иерархические категории

In [9]:
hier_cols = ["management_group_id", "first_category_id", "second_category_id", "third_category_id"]

cat_summary = pd.DataFrame({
    "train_unique": [train[c].nunique() for c in hier_cols],
    "test_unique": [test[c].nunique() for c in hier_cols],
}, index=hier_cols)
display(cat_summary)
print()

# Проверка изменчивости категорий по товарам
var_summary = []
for col in hier_cols:
    uniq_per_product = train.groupby("product_id")[col].nunique()
    var_summary.append({
        "feature": col,
        "share_products_with_multiple_values": (uniq_per_product > 1).mean(),
        "median_unique_per_product": uniq_per_product.median(),
    })

var_df = pd.DataFrame(var_summary)
display(var_df)

,train_unique,test_unique
management_group_id,7,7
first_category_id,29,29
second_category_id,76,76
third_category_id,197,197


,feature,share_products_with_multiple_values,median_unique_per_product
0,management_group_id,0.851546,3.000000
1,first_category_id,0.969072,2.000000
2,second_category_id,1.000000,34.000000
3,third_category_id,1.000000,2.000000


Иерархические категории меняются по датам для большинства товаров. Это означает, что их нужно рассматривать как динамические признаки, а не фиксированные характеристики товара. В модели используется Target Encoding для всех уровней иерархии.

## Выводы

1. Данные чистые, без пропусков и дубликатов.
2. В test 150 новых товаров, что требует специальной валидации (proxy validation).
3. Кластеризация товаров: по silhouette оптимально k=4, но для модели лучше k=5.
4. Детекция аномалий используется для выделения нестандартных товаров.
5. Наблюдается автокорреляция целей, что обосновывает сглаживание предсказаний.
6. `activity_flag` влияет на цены и используется в модели.
7. Иерархические категории динамичны и требуют Target Encoding.